# Analysis of Forcing Fields

`mom6_tools.MOM6grid` returns an object with MOM6 grid data.

`mom6_tools.latlon_analysis` has a collection of tools used to perform spatial analysis  (e.g., time averages and spatial mean).

The goal of this notebook is the following:

1) serve as an example of how to post-process CESM/MOM6 output;

2) create time averages of forcing fields;

3) create time-series of globally-averaged forcing fields;

4) compare model results vs. observations (TODO);

5) compare model results vs. another model results (TODO).

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline
import warnings, yaml, os
warnings.filterwarnings("ignore")
from mom6_tools.MOM6grid import MOM6grid
from mom6_tools.m6toolbox import genBasinMasks, cime_xmlquery
from mom6_tools.latlon_analysis import time_mean_latlon
import matplotlib.pyplot as plt
import numpy as np

Basemap module not found. Some regional plots may not function properly


ERROR 1: PROJ: proj_create_from_database: Open of /glade/work/gmarques/conda-envs/mom6-tools/share/proj failed


In [3]:
# Read in the yaml file
diag_config_yml_path = "diag_config.yml"
diag_config_yml = yaml.load(open(diag_config_yml_path,'r'), Loader=yaml.Loader)

In [4]:
caseroot = diag_config_yml['Case']['CASEROOT']
casename = cime_xmlquery(caseroot, 'CASE')
DOUT_S = cime_xmlquery(caseroot, 'DOUT_S')
rundir = cime_xmlquery(caseroot, 'RUNDIR')

if DOUT_S:
  OUTDIR = cime_xmlquery(caseroot, 'DOUT_S_ROOT')+'/ocn/hist/'
else:
  OUTDIR = cime_xmlquery(caseroot, 'RUNDIR')
    
print('Rundir directory is:', rundir)
print('Casename is:', casename)

Rundir directory is: /glade/derecho/scratch/gmarques/g.e30_a03c.GJRAv4.TL319_t232_wgx3_hycom1_N75.2024.079/run
Casename is: g.e30_a03c.GJRAv4.TL319_t232_wgx3_hycom1_N75.2024.079


In [5]:
# create an empty class object
class args:
  pass

args.case_name = casename
args.start_date = ''
args.end_date = ''
args.savefigs = False
args.time_series = True
# set avg dates
avg = diag_config_yml['Avg']
if not args.start_date : args.start_date = avg['start_date']
if not args.end_date : args.end_date = avg['end_date']
# array with 2D forcing variables to be processed
args.variables = ['hfds','wfo', 'prlq']
args.nw = 6 # number of workers
args.static = casename+diag_config_yml['Fnames']['static']
args.native = casename+diag_config_yml['Fnames']['native']
args.geom =   casename+diag_config_yml['Fnames']['geom']
args.infile = OUTDIR+args.native

In [6]:
# read grid info
geom_file = OUTDIR+'/'+args.geom
if os.path.exists(geom_file):
  grd = MOM6grid(OUTDIR+'/'+args.static, geom_file, xrformat=True)
else:
  grd = MOM6grid(OUTDIR+'/'+args.static, xrformat=True)
      
try:
  depth = grd.depth_ocean.values
except:
  depth = grd.deptho.values
    
try:
  area = grd.area_t.values
except:
  area = grd.areacello.values

MOM6 grid successfully loaded... 



In [7]:
# remove Nan's, otherwise genBasinMasks won't work
depth[np.isnan(depth)] = 0.0
basin_code = genBasinMasks(grd.geolon.values, grd.geolat.values, depth, verbose=False)

In [8]:
%matplotlib inline
# plot time averages. If variables is NOT specified, all 2D variables will be plotted.
#time_mean_latlon_plot(args,grd, variables=args.variables)
time_mean_latlon(args,grd,args.variables)

Requesting 6 workers... 

/proxy/43571/status
About to plot time-average for Surface ocean heat flux from SW+LW+latent+sensible+masstransfer+frazil+seaice_melt_heat (hfds)... 



IndexError: Unlabeled multi-dimensional array cannot be used for indexing: yh